In [1]:
%pip install beautifulsoup4
%pip install langdetect
%pip install numpy
%pip install nltk
%pip install pandas
%pip install spacy
%pip install lxml
!.pyvenv/bin/python -m spacy download en_core_web_sm

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 55.6 MB/s eta 0:00:000:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
# Import Python libraries
import os
import time
import numpy as np
import pandas as pd
import re
from bs4 import BeautifulSoup
from langdetect import detect
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.snowball import EnglishStemmer
import spacy
from spacy import displacy

In [3]:
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /home/clement/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /home/clement/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /home/clement/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /home/clement/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /home/clement/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /home/clement/nltk_data...
[nltk_data]    |   Package movie_reviews is already

True

In [4]:
data = pd.read_csv('data/QueryResults.csv')

data.set_index('Id', inplace=True)
data['CreationDate'] = pd.to_datetime(data['CreationDate'])
#data['ClosedDate'] = pd.to_datetime(data['ClosedDate'])


data.head(3)

,Title,Body,Tags,Score,ViewCount,FavoriteCount,CreationDate,AnswerCount
Id,,,,,,,,
4,How to convert Decimal to Double in C#?,<p>I want to assign the decimal variable &quot...,<c#><floating-point><type-conversion><double><...,810,78382,NaN,2008-07-31 21:42:52,13
11,Calculate relative time in C#,<p>Given a specific <code>DateTime</code> valu...,<c#><datetime><time><datediff><relative-time-s...,1656,203898,NaN,2008-07-31 23:55:37,42
13,Determine a user's timezone,<p>Is there a standard way for a web server to...,<html><browser><timezone><user-agent><timezone...,723,312544,NaN,2008-08-01 00:42:38,27


In [5]:
# Print full dataset infos
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50000 entries, 4 to 16810676
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Title          50000 non-null  object        
 1   Body           50000 non-null  object        
 2   Tags           50000 non-null  object        
 3   Score          50000 non-null  int64         
 4   ViewCount      50000 non-null  int64         
 5   FavoriteCount  0 non-null      float64       
 6   CreationDate   50000 non-null  datetime64[ns]
 7   AnswerCount    50000 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(3)
memory usage: 3.4+ MB


In [6]:
# Describe data
data.describe()

,Score,ViewCount,FavoriteCount,CreationDate,AnswerCount
count,50000.000000,5.000000e+04,0.0,50000,50000.000000
mean,12.068220,1.106723e+04,NaN,2014-11-24 13:05:39.635759616,2.202380
min,-8.000000,1.600000e+01,NaN,2008-07-31 21:42:52,1.000000
25%,0.000000,3.680000e+02,NaN,2011-11-19 20:09:04.750000128,1.000000
50%,1.000000,1.084000e+03,NaN,2014-09-24 14:06:40,1.000000
75%,3.000000,3.119000e+03,NaN,2017-08-19 09:05:04,2.000000
max,9463.000000,4.865464e+06,NaN,2021-08-18 22:31:13,91.000000
std,151.132812,9.946791e+04,NaN,NaN,2.920882


In [7]:
#data = data.head(100)

In [8]:
display(data['Tags'].head(5))
data['Tags'] = (data['Tags'].str.lstrip('<').str.rstrip('>').str.split('><')).str.join(',')


Id
4     <c#><floating-point><type-conversion><double><...
11    <c#><datetime><time><datediff><relative-time-s...
13    <html><browser><timezone><user-agent><timezone...
19    <performance><algorithm><language-agnostic><un...
25                    <c++><c><sockets><mainframe><zos>
Name: Tags, dtype: object

In [9]:
data['Code'] = data['Body'].apply(lambda x: '\n'.join(re.findall(r'<code>(.*?)</code>', x)))
data['Body'] = data['Body'].apply(lambda x: re.sub(r'<code>(.*?)</code>', '', x))
data['Body'] = data['Body'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())

display(data['Body'].head(3))

data['Code'].head(50)


Id
4     I want to assign the decimal variable "trans" ...
11    Given a specific  value, how do I display rela...
13    Is there a standard way for a web server to be...
Name: Body, dtype: object

Id
4                                                            
11             DateTime\n2 hours ago\n3 days ago\na month ago
13                                                 user-agent
19          #define\nM_PI\n4 * atan(1)\natan(1)\n-fno-buil...
25          sys/socket.h\nAF_INET\n#ifdef\n_OE_SOCKETS\n#i...
7998733                                                      
25992632                                                     
16493577    array('select' =&gt; array('label' =&gt; "elem...
7998773                                                      
88          QueryPerformanceCounter\nQueryPerformanceFrequ...
25992619                                                     
109                                                          
25992683                                                     
129                                           SQL Server 2005
25992706                                                     
7998816     emb\nimport emb\nemb\n__import__()\n__import__...
146  

In [10]:
nlp = spacy.load('en_core_web_sm')
stop_words = stopwords.words("english")
wn = WordNetLemmatizer()

In [11]:
def filter_and_lemmatize_tokens(doc):
 # Filtrer les tokens pour ne garder que les noms et les noms propres
    filtered = filter(lambda token: token.pos_ in ["NOUN", "PROPN"], doc)
    
    # Lemmatiser les tokens filtrés
    lemmatized = list(map(lambda token: token.lemma_, filtered))

    lemmatized = list(set(lemmatized))
    
    # Normaliser les tokens
    normalized_text = ' '.join(lemmatized).lower()
    normalized_text = normalized_text.encode("ascii", "ignore").decode()
    normalized_text = re.sub("\'\w+", '', normalized_text)
    normalized_text = re.sub('[^\\w\\s#]', '', normalized_text)
    normalized_text = re.sub(r'http*\S+', '', normalized_text)
    normalized_text = re.sub(r'\w*\d+\w*', '', normalized_text)
    normalized_text = re.sub('\s+', ' ', normalized_text)
    
    return normalized_text

In [12]:
data = data[data['Body'].apply(lambda x: detect(x[0:100])) == 'en']


data['Body'] = data['Body'].apply(lambda x: filter_and_lemmatize_tokens(nlp(x)))
data['Title'] = data['Title'].apply(lambda x: filter_and_lemmatize_tokens(nlp(x)))


In [13]:
# Export to CSV
data.to_csv("data/StackOverflow_tokenized.csv", sep=";")